In [1]:
import torch
from transformers import RobertaForMultipleChoice
import logging
logging.basicConfig(level=logging.INFO)
from tqdm import tqdm
import numpy as np
import os
import pandas as pd
from torch.utils.data import DataLoader, Dataset

In [2]:
import torch
cuda_id = 0
device = torch.device("cuda:%s" % cuda_id if torch.cuda.is_available() else "cpu")
device_name = torch.cuda.get_device_name(cuda_id) if torch.cuda.is_available() else "cpu"
print("We are using the device %s - %s" % (device, device_name))
model_name = "../../roberta-large"
learning_rate = 1.5e-5
batch_size = 24
epoch = 8
optimizer_name = 'Adam'

We are using the device cuda:0 - GeForce RTX 2080 Ti


In [3]:
def get_info_from_csv(texts_path, labels_path):
    texts = pd.read_csv(texts_path, header=0, names=['ID', 'Sentence 0', 'Sentence 1'])
    labels = pd.read_csv(labels_path, header=None, names=['ID', 'Answer'])['Answer']
    return texts, labels

In [4]:
train_texts, train_labels = get_info_from_csv(
    '../../DataSet/Training Data/subtaskA_data_all.csv',
    '../../DataSet/Training Data/subtaskA_answers_all.csv'
)

val_texts, val_labels = get_info_from_csv(
    '../../DataSet/Dev Data/subtaskA_dev_data.csv',
    '../../DataSet/Dev Data/subtaskA_gold_answers.csv'
)

test_texts, test_labels = get_info_from_csv(
    '../../DataSet/Test Data/subtaskA_test_data.csv',
    '../../DataSet/Test Data/subtaskA_gold_answers.csv'
)

# Use the validation dataset as the training dataset
train_texts = pd.concat([train_texts, val_texts], axis=0, ignore_index=True)
train_labels = pd.concat([train_labels, val_labels], axis=0, ignore_index=True)
train_data = pd.concat([train_texts, train_labels], axis=1)
train_data

ID                                         Sentence 0  \
0         0              He poured orange juice on his cereal.   
1         1                                   He drinks apple.   
2         2                              Jeff ran a mile today   
3         3                               A mosquito stings me   
4         4                               A niece is a person.   
...     ...                                                ...   
10992   700                       THE TEMPERATURE IS EMOTIONAL   
10993   355                     Blended up fruit makes a nacho   
10994  1550                            Dogs falls from the sky   
10995  1364                    He put a dinosaur in the fridge   
10996  1257  The cleaner is in charge of the money at the s...   

                                              Sentence 1  Answer  
0                          He poured milk on his cereal.       0  
1                                        He drinks milk.       0  
2                           Jeff ran 100,000 miles today       1  
3                                     I sting a mosquito       1  
4                                 A giraffe is a person.       1  
...                                                  ...     ...  
10992                            THE TEMPERATURE IS WARM       0  
10993                  Blended up fruit makes a smoothie       0  
10994                            Rain falls from the sky       0  
10995                     He put a chicken in the fridge       0  
10996  The cashier is in charge of the money at the s...       0  

[10997 rows x 4 columns]

In [5]:

from transformers import  RobertaTokenizerFast,AdamW

tokenizer = RobertaTokenizerFast.from_pretrained(model_name)


train_encodings = tokenizer(sum([train_texts.iloc[i, 1:].tolist() for i in range(len(train_texts))], start=[]),
[''] * 2 * len(train_texts),return_tensors="pt", 
                            padding=True)
train_encodings = {key: value.view(len(train_texts), 2, -1) for key, value in train_encodings.items()}

val_encodings = tokenizer(sum([val_texts.iloc[i, 1:].tolist() for i in range(len(val_texts))], start=[]), 
[''] * 2 * len(val_texts),
                    return_tensors="pt", 
                            padding=True)
val_encodings = {key: value.view(len(val_texts), 2, -1) for key, value in val_encodings.items()}

test_encodings = tokenizer(sum([test_texts.iloc[i, 1:].tolist() for i in range(len(test_texts))], start=[]),
[''] * 2 * len(test_texts) ,return_tensors="pt", 
                            padding=True)
test_encodings = {key: value.view(len(test_texts), 2, -1) for key, value in test_encodings.items()}

In [6]:
class ComVEDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [7]:
train_dataset = ComVEDataset(train_encodings, train_labels)
test_dataset = ComVEDataset(test_encodings, test_labels)
val_dataset = ComVEDataset(val_encodings, val_labels)

In [8]:
model = RobertaForMultipleChoice.from_pretrained(model_name).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

Some weights of the model checkpoint at ../../roberta-large were not used when initializing RobertaForMultipleChoice: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultipleChoice were not initialized from the model checkpoint at ../../roberta-large and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to us

In [9]:
# Prediction function
def predict(outputs):
    probabilities = torch.softmax(outputs["logits"], dim=1)
    predictions = torch.argmax(probabilities, dim=1)
    return predictions
# Loading the optimizer
if optimizer_name == "Adam":
    optim = AdamW(model.parameters(), lr=learning_rate)

/root/miniconda/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [10]:
def validate(model):

    # Get test data by DataLoader
    val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
    # Start testing
    model.eval()

    with torch.no_grad():
        
        correct = 0
        count = 0
        record = list()
        
        pbar = tqdm(val_loader)
        for batch in pbar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs['loss']
            
            # make predictions
            predictions = predict(outputs)

            # count accuracy
            correct += predictions.eq(labels).sum().item()
            count += len(labels)
            accuracy = correct * 1.0 / count

            # show progress along with metrics
            pbar.set_postfix({
                'loss': '{:.3f}'.format(loss.item()),
                'accuracy': '{:.3f}'.format(accuracy)
            })
        
            # record the results
            record.append((int(labels),int(predictions)))
            
        pbar.close()
    time.sleep(0.3)
    print("The final accuracy on the validate dataset: %s%%" % round(accuracy*100,4))
    return round(accuracy*100,4)

In [11]:
# Training
from torch.utils.data import DataLoader
import time
# Set the number of epoch
epoch = epoch

# Get training data by DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Start training
model.train()

train_loss = list()
train_accuracies = list()
highest_accuracy = 0
for epoch_i in range(epoch):
    print('Epoch %s/%s' % (epoch_i + 1, epoch))
    time.sleep(0.3)
    
    correct = 0
    count = 0
    epoch_loss = list()
    
    pbar = tqdm(train_loader)
    for batch in pbar:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs['loss']
        loss.backward()
        optim.step()
        
        # make predictions
        predictions = predict(outputs)

        # count accuracy
        correct += predictions.eq(labels).sum().item()
        count += len(labels)
        accuracy = correct * 1.0 / count

        # show progress along with metrics
        pbar.set_postfix({
            'Loss': '{:.3f}'.format(loss.item()),
            'Accuracy': '{:.3f}'.format(accuracy)
        })
        
        # record the loss for each batch
        epoch_loss.append(loss.item())
        
    pbar.close()
    
    # record the loss and accuracy for each epoch
    train_loss += epoch_loss
    train_accuracies.append(accuracy)
    val_accuracy=validate(model)
    
    # save the model with the highest accuracy
    if highest_accuracy <= val_accuracy:
        highest_accuracy = val_accuracy
    torch.save(model.state_dict(), "./Subtask_A_"+str( epoch_i)+"_Model.pth")

Epoch 1/8


  0%|          | 0/459 [00:00<?, ?it/s]/tmp/ipykernel_599/3522555481.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 997/997 [00:33<00:00, 30.15it/s, loss=0.693, accuracy=0.743]


The final accuracy on the validate dataset: 74.323%
Epoch 2/8


100%|██████████| 997/997 [00:30<00:00, 32.24it/s, loss=0.693, accuracy=0.711]


The final accuracy on the validate dataset: 71.1133%
Epoch 3/8


...

The final accuracy on the validate dataset: 100.0%
Epoch 7/8


100%|██████████| 997/997 [00:33<00:00, 29.58it/s, loss=0.000, accuracy=1.000]


The final accuracy on the validate dataset: 100.0%
Epoch 8/8


100%|██████████| 997/997 [00:35<00:00, 28.30it/s, loss=0.000, accuracy=1.000]


The final accuracy on the validate dataset: 100.0%


In [12]:
torch.cuda.empty_cache()

In [15]:
def Test(model_params_path):   # Testing

    # Load the best-trained model
    model.load_state_dict(torch.load(model_params_path))

    # Get test data by DataLoader
    test_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

    # Start testing
    model.eval()

    with torch.no_grad():
        
        correct = 0
        count = 0
        record = list()
        
        pbar = tqdm(test_loader)
        for batch in pbar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs['loss']
            
            # make predictions
            predictions = predict(outputs)

            # count accuracy
            correct += predictions.eq(labels).sum().item()
            count += len(labels)
            accuracy = correct * 1.0 / count

            # show progress along with metrics
            pbar.set_postfix({
                'loss': '{:.3f}'.format(loss.item()),
                'accuracy': '{:.3f}'.format(accuracy)
            })
        
            # record the results
            record.append((int(labels),int(predictions)))
            
        pbar.close()
        
    time.sleep(0.3)
    print("The final accuracy on the test dataset: %s%%" % round(accuracy*100,4))
    
#for epoch_i in range(epoch):
#    print("epoch: ", epoch_i)
#    Test("./Subtask_A_"+str( epoch_i)+"_Model.pth")
Test("./O2_roberta_Model_1.pth")

  0%|          | 0/997 [00:00<?, ?it/s]/tmp/ipykernel_599/3522555481.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 997/997 [00:32<00:00, 30.55it/s, loss=0.000, accuracy=1.000]


The final accuracy on the test dataset: 100.0%
